Image Segmentation

the work apply the Generative Adversarial Network, GAN

In [ ]:
#We will download the database from https. 
#MAC computers need to add the following two lines so that the other party’s ssl certificate will not be considered invalid
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

Import library.

1.This random module implements pseudo-random number generators for various distributions.

2.Using the Torchvision resource library, we can quickly use common datasets that do not need to be processed by ourselves, use the convenient and effective image transform function to process our dataset, and use the pre-trained good model

3.PIL is Python image library. It consists of many different modules and provides many processing functions, allowing us to process images in a simple Python program. Using a library package like PIL allows us to focus on the image processing itself and avoid getting lost in the underlying algorithms.

4.matplotlib is visualization kit for Python; torch converts the model to Torch Script;the os module provides a very rich method to handle files and directories;numpy has functions for working in domain of linear algebra, fourier transform, and matrices; cv2 provides the basic image manipulation functions.

5.Load all downloader files in the current directory as modules

In [ ]:
import random
from torchvision import transforms as T
from PIL import Image
import matplotlib.pyplot as plt
import torch
import os
import numpy as np
import cv2
from downloader import *
%matplotlib inline

In [ ]:
#import Generator
from model import Generator

In [ ]:
#Use CelebA dataset
dataset ='CelebA'
#set the photo size as 256 
image_size =256
#CelebA's selected attritubes
selected_attrs =['Gray_Hair', 'Wavy_Hair', 'Pale_Skin', 'Narrow_Eyes', 'Oval_Face']
# select 5 dimensions 
c_dim =5
#choose the directories
celeba_image_dir ='data/custom/images'
model_save_dir ='stargan_celeba_256/models'
# The path to download the G weights file.
G_url = 'https://github.com/hanyoseob/pytorch-StarGAN'
# Number of conv filters in the first layer of G.
g_conv_dim = 64
# number of residual blocks in G
g_repeat_num = 6
# test model at the stage
test_iters = 200000

In [ ]:
# increase the speed of deep learning operations
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

GAN trains two neural networks at the same time, the generator (Generator, G) and the discriminator (Discriminator, D), which are stronger and stronger against each other.

In [ ]:
#Generator
G = Generator(g_conv_dim, c_dim, g_repeat_num)

In [ ]:
#network information
def print_network(model, name):
    num_params = 0
    for p in model.parameters():
        num_params += p.numel()
    print(model)
    print(name)
    print("The number of parameters: {}".format(num_params))

In [ ]:
print_network(G, 'G')

In [ ]:
G.to(device)
print('Models moved to', device)

In [ ]:
#resume training from this step
resume_iters = test_iters
print('Loading the trained models from step {}...'.format(resume_iters))
#generator path
G_path = os.path.join(model_save_dir, '{}-G.ckpt'.format(resume_iters))
download_if_not_exists(G_path, G_url)
# D_path = os.path.join(model_save_dir, '{}-D.ckpt'.format(resume_iters))
G.load_state_dict(torch.load(G_path, map_location=lambda storage, loc: storage))
# D.load_state_dict(torch.load(D_path, map_location=lambda storage, loc: storage))

In [ ]:
#the directory of case path
CASE_PATH = 'data/custom/haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(CASE_PATH)

crop-face function, set the margin and image size as celebA as 256

    :param imgarray: full image
    :param section: face detected area (x, y, w, h)
    :param margin: add some margin to the face detected area to include a full head
    :param size: the result image resolution with be (size x size)
    :return: resized image in numpy array with shape (size x size x 3)

In [ ]:
def crop_face(imgarray, section, margin=20, size=256):
    img_h, img_w, _ = imgarray.shape
    if section is None:
        section = [0, 0, img_w, img_h]
    (x, y, w, h) = section
    margin = int(min(w, h) * margin / 100)
    x_a = x - margin
    y_a = y - margin
    x_b = x + w + margin
    y_b = y + h + margin
    if x_a < 0:
        x_b = min(x_b - x_a, img_w - 1)
        x_a = 0
    if y_a < 0:
        y_b = min(y_b - y_a, img_h - 1)
        y_a = 0
    if x_b > img_w:
        x_a = max(x_a - (x_b - img_w), 0)
        x_b = img_w
    if y_b > img_h:
        y_a = max(y_a - (y_b - img_h), 0)
        y_b = img_h
    cropped = imgarray[y_a: y_b, x_a: x_b]
    resized_img = cv2.resize(cropped, (size, size), interpolation=cv2.INTER_AREA)
    resized_img = np.array(resized_img)
    return resized_img

In [ ]:
def tryCvStarGAN(imgFile, setHairColor = 'Gray', setNarrow_Eyes = False, setOval_Face = True):
    frame = cv2.imread(imgFile)
    frame = frame[:, :, ::-1] # CV2'BGR -> RGB
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=10
    )
    if len(faces) is 0:
        print('No face detected, try another image.')
        return
    main_face = max(faces, key=lambda rectangle: (rectangle[2] * rectangle[3]))  # area = w * h
    face_img = crop_face(frame, main_face, margin = 40, size=256)
    image = Image.fromarray(face_img)
    
    # Pre-process the image
    transform = []
    transform.append(T.ToTensor())
    transform.append(T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)))
    transform = T.Compose(transform)

    preprocessed_image = transform(face_img)
    
    # See the pre-processed source image.
    # Set the plot
    fig=plt.figure(figsize=(8, 8))
    columns = 2
    rows = 1
    fig.add_subplot(rows, columns, 1)
    plt.imshow((np.moveaxis(preprocessed_image.numpy(),[0], [2])+1)/2)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # Pre-process the image
    
    
    # Three hair color to choose from.
    hairColors = ['Black_Hair', 'Blond_Hair', 'Brown_Hair']
    labels = torch.zeros(c_dim)
    # Match the user set hair color and set the label value.
    for i, color in enumerate(hairColors):
        if setHairColor.lower() in color.lower():
            labels[i] = 1
            break
    # Set the 'Narrow_Eyes' label.
    if setNarrow_Eyes is True:
        labels[3] = 1
    # Set the 'Oval_Face' label.
    if setOval_Face is True:
        labels[4] = 1
    # the generator to generate the image with labels that choosed.
    generated = G(preprocessed_image.unsqueeze(0).to(device), labels.unsqueeze(0).to(device))
    
    # use matplotlib to see the image from generator.
    fig.add_subplot(rows, columns, 2)
    plt.imshow((np.moveaxis(generated.cpu().detach().numpy()[0],[0], [2])+1)/2)
    plt.show()


In [ ]:
tryCvStarGAN('data/custom/images/000016.jpg', setHairColor = 'brown', setNarrow_Eyes = False, setOval_Face = False)

In [ ]:
tryCvStarGAN('data/custom/images/000016.jpg', setHairColor = 'blond', setNarrow_Eyes = False, setOval_Face = False)

In [ ]:
t2 = torch.ones(2)
t2.numpy

In [ ]:
torch.ones(2).numpy